In [3]:
import setup_django
setup_django.init()

In [4]:
import pandas as pd
from django.conf import settings
from movies.models import Movie
from ratings.models import Rating

LINKS_SMALL_CSV = settings.DATA_DIR / 'links_small.csv'
LINKS_SMALL_CSV.exists()

True

In [5]:
qs = Rating.objects.all()
missing_movie_ids = []
for instance in qs:
    if instance.content_object is None:
        missing_movie_ids.append(instance.object_id)

_total = len(missing_movie_ids)
total_missing = list(set(missing_movie_ids))

In [6]:
print(len(total_missing), _total, qs.count())

9064 99986 100004


In [7]:
links_df = pd.read_csv(LINKS_SMALL_CSV)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
ms_df = links_df.copy()[links_df.movieId.isin(total_missing)]
ms_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [9]:
ms_df.shape[0] == len(total_missing)

True

In [10]:
def enrich_imdb_col(val):
    val = str(val)
    if len(val) == 7:
        val = f"tt{val}"
        return val
    if len(val) == 6:
        val = f"tt0{val}"
        return val
    if len(val) == 5:
        val = f"tt00{val}"
        return val
    return val

In [11]:
ms_df['tt'] = ms_df['imdbId'].apply(enrich_imdb_col)
ms_df.head()

,movieId,imdbId,tmdbId,tt
0,1,114709,862.0,tt0114709
1,2,113497,8844.0,tt0113497
2,3,113228,15602.0,tt0113228
3,4,114885,31357.0,tt0114885
4,5,113041,11862.0,tt0113041


In [12]:
MOVIES_CSV = settings.DATA_DIR / 'movies_metadata.csv'
MOVIES_CSV.exists()

True

In [13]:
movies_cols = ['title', 'overview', 'release_date', 'imdb_id']
movies_df = pd.read_csv(MOVIES_CSV, usecols=movies_cols)
movies_df.head()

,imdb_id,overview,release_date,title
0,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [14]:
missing_movies_df = ms_df.merge(movies_df, left_on='tt', right_on='imdb_id')
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,3,113228,15602.0,tt0113228,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,5,113041,11862.0,tt0113041,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [15]:
missing_movies_df['id'] = missing_movies_df['movieId']
missing_movies_df['id_alt'] = missing_movies_df['tmdbId'].apply(lambda x: str(int(x)))
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title,id,id_alt
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story,1,862
1,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,2,8844
2,3,113228,15602.0,tt0113228,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men,3,15602
3,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale,4,31357
4,5,113041,11862.0,tt0113041,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II,5,11862


In [16]:
final_df = missing_movies_df.copy()[['id', 'id_alt', 'title']]
final_df['id_alt'] = final_df['id_alt'].astype(str)
final_df.head()

,id,id_alt,title
0,1,862,Toy Story
1,2,8844,Jumanji
2,3,15602,Grumpier Old Men
3,4,31357,Waiting to Exhale
4,5,11862,Father of the Bride Part II


In [17]:
alt_id_list = final_df['id_alt'].to_list()

In [18]:
movies_qs = Movie.objects.all()
movies_qs.count()

11

In [19]:
from django.forms.models import model_to_dict

In [20]:
movies_qs = Movie.objects.filter(id__in=alt_id_list)
for obj in movies_qs:
    data = final_df.copy()[final_df['id_alt'] == str(obj.id)]
    if data.shape[0] == 1:
        og_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]
        if obj.title == update_data.get('title'): 
            # print(og_model_data)
            og_model_data['id'] = update_data['id']
            new_model_data = {**og_model_data}
            # print(new_model_data)
            obj.delete()
            Movie.objects.create(**new_model_data)

In [21]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

Rating update took 0:00:06 (6.2293922901153564s)
